### **Group**: Peter Endes-Nagy, Khawaja Hassan, Shah Ali, Viktoria Konya


# **Task2**

Use the senator speeches in the folder 105-extracted-date located in Inputs and do the following:

i) text preprocessing, removing stopwords, punctuations. Justify your choices.

ii) use cosine similarity to find whose senator's speech is closest to senator Biden. Use sen105kh_fix.csv and/or Wikipedia to validate your findings (i.e., understand if the most similar speeches are senators from the same state and/or party). Describe your findings. 

iv) How do your results change if you apply stemming or lemmatization? In your opinion is it better to apply stemming or lemmatization? Why?

In [1]:
# Import libraries
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Change path
path_input = "Inputs/105-extracted-date"

Please make sure you have the right path and you're in the Python-Programming-and-Text-Analysis folder

In [3]:
#my_path = ''
#os.chdir(my_path)
os.getcwd()

'/Users/endesnagypeter/Documents/CEU/Python_text/Python-Programming-and-Text-Analysis'

## 1. Data import

In [4]:
# Read in files and create df
speech = []
filename = []
for element in sorted(os.listdir(path_input)):
    if ".txt" in element:
        file = open(os.path.join(path_input,element) , errors = 'ignore' ).readlines() 
        file = ''.join(file) 
        filename.append(element.replace(".txt", ""))
        speech.append(file)

# Put messages to dataframe
df = pd.DataFrame({'filename':filename,'speech':speech})

# Check
pd.set_option("display.max_columns", None)
df[0:10]

,filename,speech
0,105-abraham-mi,<DOC>\n<DOCNO>105-abraham-mi-1-19981112</DOCNO...
1,105-akaka-hi,<DOC>\n<DOCNO>105-akaka-hi-1-19981021</DOCNO>\...
2,105-allard-co,<DOC>\n<DOCNO>105-allard-co-1-19981009</DOCNO>...
3,105-ashcroft-mo,<DOC>\n<DOCNO>105-ashcroft-mo-1-19981021</DOCN...
4,105-baucus-mt,<DOC>\n<DOCNO>105-baucus-mt-1-19981021</DOCNO>...
5,105-bennett-ut,<DOC>\n<DOCNO>105-bennett-ut-1-19981112</DOCNO...
6,105-biden-de,<DOC>\n<DOCNO>105-biden-de-1-19981021</DOCNO>\...
7,105-bingaman-nm,<DOC>\n<DOCNO>105-bingaman-nm-1-19981021</DOCN...
8,105-bond-mo,<DOC>\n<DOCNO>105-bond-mo-1-19981012</DOCNO>\n...
9,105-boxer-ca,<DOC>\n<DOCNO>105-boxer-ca-1-19981021</DOCNO>\...


## 2. Data preparation

First, we need to retrieve only the text from the files.

For this we are going to:
* Parse the documents with BeautifulSoup
* Remove the words that contain the filename
* Do additional data cleaning: removeing the \n, removing extra spaces and leading spaces.

In [5]:
# Define function to remove the filenames
def replaceName(document, filename):
    document_clean = re.sub(r"(?:\s){}[^, ]*".format(filename), "", document) 
    return str(document_clean)

In [6]:
my_dict = dict.fromkeys(df['filename'])

for ind in df.index:
    
    # Get filename
    filename = df['filename'][ind]
    
    # Get speech
    speech = df['speech'][ind]
    
    # Parse speech
    soup = BeautifulSoup(speech, 'html')
    
    # Remove \n
    s = soup.text.replace("\n", " ")

    # Remove extra spaces
    s = re.sub("\s\s+", " ", s)
    
    # Remove word that contains filename
    s = replaceName(s, filename)
    
    # Remove leading spaces
    s.strip()
    
    my_dict[filename] = s

In [7]:
# Check one element
my_dict['105-abraham-mi'][:300]

' Mr. ABRAHAM. Mr. President, during debate on final passage of the Omnibus Appropriations bill, in which the American Competitiveness and Workforce Improvement Act was included as Title IV of Subdivision C, I asked unanimous consent to have a number of documents printed in the Record. These included'

In [8]:
# Convert back to df
df = pd.DataFrame.from_dict(my_dict, orient='index').rename_axis('filename').reset_index()
df.rename(columns={ df.columns[1]: "speech" }, inplace = True)
df.head()

,filename,speech
0,105-abraham-mi,"Mr. ABRAHAM. Mr. President, during debate on ..."
1,105-akaka-hi,"Mr. AKAKA. Mr. President, I am pleased that t..."
2,105-allard-co,"Mr. ALLARD. Mr. President, I rise to make a f..."
3,105-ashcroft-mo,"Mr. ASHCROFT. Mr. President, the Senate is no..."
4,105-baucus-mt,Mr. BAUCUS. I understand that the House has s...


## 3. Data preprocessing and tokenization, applying TF-IDF vectorizer, calculationg cosine similarity

In the next step the following were done:

* We created **text_preprocesser()** function that removes all punctuations, non-alphabetical characeters, stopwords and words with less than 3 characters.
* We applyed the preprocessor on the speeches.
* Then we applied the **TF-IDF vectorizer** where we choped off the lest frequent terms (that appeared in less than 5% of the documents) and the most important ters (that appeared in more than 95% of the documents).

#### 3.1. Text preprocesser

In [9]:
# Define preprocesser
def text_preprocesser(text):
    
    # Remove not alphanumerical characters
    text = re.sub(r'\W',' ', text)
    
    # Remove not alphanumerical characters
    text = re.sub("[^a-zA-Z]+", " ", text)
    
    # Tokenize
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Remove words with length less than 3 characters
    tokens = [word for word in tokens if len(word)>=3]
    
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [10]:
# Check one element
text_preprocesser(df['speech'].iloc[0])[:300]

'abraham president debate final passage omnibus appropriations bill american competitiveness workforce improvement act included title subdivision asked unanimous consent number documents printed record included two documents received administration negotiations whose inclusion seeking help illuminate'

#### 3.2. Apply text preprocesser on the speeches

As it's a time-consuming process, the result were saved, then read back. Please uncomment the code chunk for testing the code. 

In [11]:
##### Apply to df

#df_preprocessed = df['speech'].apply(text_preprocesser)

#### Save

#import pickle
#with open(f'df_preprocessed.p', 'wb') as f:
        #pickle.dump(df_preprocessed, f)

In [12]:
# Import csv again
import pickle
with open(f'df_preprocessed.p', 'rb') as data:
    df_preprocessed = pickle.load(data)

df_preprocessed[0][:300]

'abraham president debate final passage omnibus appropriations bill american competitiveness workforce improvement act included title subdivision asked unanimous consent number documents printed record included two documents received administration negotiations whose inclusion seeking help illuminate'

In [13]:
len(df_preprocessed)

100

#### 3.3. Apply TF-IDF vectorizer

We will:
* ignore terms that appear in less than 5% of the documents and
* ignore terms that appear in more than 95% of the documents

In [14]:
# Apply TF-IDF vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=0.05, max_df= 0.95) # preprocessing was done before
tfidf = tfidf_vectorizer.fit_transform(df_preprocessed)

In [15]:
# Convert to df
df_tfidf = pd.DataFrame(tfidf.toarray().transpose(), index=tfidf_vectorizer.get_feature_names())
df_tfidf.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
aaa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.009506,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005864,0.000000,0.00875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aapcc,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010124,0.000000,0.003027,0.000000,0.004333,0.007435,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.020153,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.016850,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.056279
aaron,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009087,0.002231,0.000000,0.000000,0.001944,0.000000,0.000000,0.001355,0.000000,0.001732,0.000000,0.000000,0.001886,0.000000,0.000000,0.000000,0.000000,0.001658,0.0,0.002737,0.000000,0.006041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002162,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001475,0.002989,0.003962,0.000902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001349,0.000000,0.001498,0.000000,0.000000,0.000000,0.003272,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.013637,0.004084,0.002567,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aarp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002205,0.000000,0.000000,0.033106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002536,0.000000,0.005559,0.000000,0.011108,0.002856,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001865,0.000000,0.000000,0.000000,0.003438,0.0000,0.004516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000956,0.000000,0.000000,0.000000,0.002333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005289,0.000000,0.003248,0.000000,0.002368,0.000000,0.0,0.000000,0.000000,0.002085,0.000000,0.003883,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aba,0.004530,0.000000,0.000000,0.004474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [16]:
# Column names
senator_names = df['filename'].str.split('-').str[1] + '-' + df['filename'].str.split('-').str[2] 
senator_names.head()

0     abraham-mi
1       akaka-hi
2      allard-co
3    ashcroft-mo
4      baucus-mt
Name: filename, dtype: object

In [17]:
# Assign column names to matrix
df_tfidf.columns = senator_names
df_tfidf.head(10)

filename,abraham-mi,akaka-hi,allard-co,ashcroft-mo,baucus-mt,bennett-ut,biden-de,bingaman-nm,bond-mo,boxer-ca,breaux-la,brownback-ks,bryan-nv,bumpers-ar,burns-mt,byrd-wv,campbell-co,chafee-ri,cleland-ga,coats-in,cochran-ms,collins-me,conrad-nd,coverdell-ga,craig-id,damato-ny,daschle-sd,dewine-oh,dodd-ct,domenici-nm,dorgan-nd,durbin-il,enzi-wy,faircloth-nc,feingold-wi,feinstein-ca,ford-ky,frist-tn,glenn-oh,gorton-wa,graham-fl,gramm-tx,grams-mn,grassley-ia,gregg-nh,hagel-ne,harkin-ia,hatch-ut,helms-nc,hollings-sc,hutchinson-ar,hutchison-tx,inhofe-ok,inouye-hi,jeffords-vt,johnson-sd,kempthorne-id,kennedy-ma,kerrey-ne,kerry-ma,kohl-wi,kyl-az,landrieu-la,lautenberg-nj,leahy-vt,levin-mi,lieberman-ct,lott-ms,lugar-in,mack-fl,mccain-az,mcconnell-ky,mikulski-md,moseleybraun-il,moynihan-ny,murkowski-ak,murray-wa,nickles-ok,reed-ri,reid-nv,robb-va,roberts-ks,rockefeller-wv,roth-de,santorum-pa,sarbanes-md,sessions-al,shelby-al,smith-nh,smith-or,snowe-me,specter-pa,stevens-ak,thomas-wy,thompson-tn,thurmond-sc,torricelli-nj,warner-va,wellstone-mn,wyden-or
aaa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027523,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.009506,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.005864,0.000000,0.00875,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aapcc,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010124,0.000000,0.003027,0.000000,0.004333,0.007435,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.020153,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.016850,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.056279
aaron,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009087,0.002231,0.000000,0.000000,0.001944,0.000000,0.000000,0.001355,0.000000,0.001732,0.000000,0.000000,0.001886,0.000000,0.000000,0.000000,0.000000,0.001658,0.0,0.002737,0.000000,0.006041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002162,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001475,0.002989,0.003962,0.000902,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001349,0.000000,0.001498,0.000000,0.000000,0.000000,0.003272,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.013637,0.004084,0.002567,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aarp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002205,0.000000,0.000000,0.033106,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002536,0.000000,0.005559,0.

In [18]:
len(df_tfidf)

22530

#### 4. Apply cosine similarity

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
# Get number of columns
num_col = len(df_tfidf.columns)
num_col

100

In [21]:
# Reshape 
for i in range(1, num_col+1):
    globals()["txt" + str(i)] = df_tfidf[df_tfidf.columns[i-1]].values.reshape(1, -1) 

In [22]:
# Biden
txt7

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00168191]])

In [23]:
# Similarity between Biden and Roberts - sample
print("Similarity Biden and Roberts:", cosine_similarity(txt7, txt82))

Similarity Biden and Roberts: [[0.39314126]]


In [24]:
# Reshape to vector
import numpy as np
for i in range(1, num_col+1):
    globals()["txt" + str(i)] = np.squeeze(np.asarray(globals()["txt" + str(i)]))

In [25]:
txt7

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00168191])

In [26]:
# Create list of vectors
list_of_txts = []
for i in range(1, num_col+1):
    element = "txt" + str(i)
    list_of_txts.append(element)
print(list_of_txts)

['txt1', 'txt2', 'txt3', 'txt4', 'txt5', 'txt6', 'txt7', 'txt8', 'txt9', 'txt10', 'txt11', 'txt12', 'txt13', 'txt14', 'txt15', 'txt16', 'txt17', 'txt18', 'txt19', 'txt20', 'txt21', 'txt22', 'txt23', 'txt24', 'txt25', 'txt26', 'txt27', 'txt28', 'txt29', 'txt30', 'txt31', 'txt32', 'txt33', 'txt34', 'txt35', 'txt36', 'txt37', 'txt38', 'txt39', 'txt40', 'txt41', 'txt42', 'txt43', 'txt44', 'txt45', 'txt46', 'txt47', 'txt48', 'txt49', 'txt50', 'txt51', 'txt52', 'txt53', 'txt54', 'txt55', 'txt56', 'txt57', 'txt58', 'txt59', 'txt60', 'txt61', 'txt62', 'txt63', 'txt64', 'txt65', 'txt66', 'txt67', 'txt68', 'txt69', 'txt70', 'txt71', 'txt72', 'txt73', 'txt74', 'txt75', 'txt76', 'txt77', 'txt78', 'txt79', 'txt80', 'txt81', 'txt82', 'txt83', 'txt84', 'txt85', 'txt86', 'txt87', 'txt88', 'txt89', 'txt90', 'txt91', 'txt92', 'txt93', 'txt94', 'txt95', 'txt96', 'txt97', 'txt98', 'txt99', 'txt100']


In [27]:
# Create similarity matrix
from scipy import sparse
A =  np.array(np.array([eval(name) for name in list_of_txts]))
A_sparse = sparse.csr_matrix(A)
similarities = cosine_similarity(A_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.         0.11286872 0.1361684  ... 0.09720429 0.16259734 0.13949189]
 [0.11286872 1.         0.07545337 ... 0.04603068 0.07772243 0.06784303]
 [0.1361684  0.07545337 1.         ... 0.0607008  0.09026423 0.08167521]
 ...
 [0.09720429 0.04603068 0.0607008  ... 1.         0.08235814 0.08108126]
 [0.16259734 0.07772243 0.09026423 ... 0.08235814 1.         0.13055899]
 [0.13949189 0.06784303 0.08167521 ... 0.08108126 0.13055899 1.        ]]



In [28]:
# Define similarity matrix
df_similarity = pd.DataFrame(similarities)
df_similarity.columns = senator_names
df_similarity.insert(0,'senator',senator_names)
df_similarity.head()

filename,senator,abraham-mi,akaka-hi,allard-co,ashcroft-mo,baucus-mt,bennett-ut,biden-de,bingaman-nm,bond-mo,boxer-ca,breaux-la,brownback-ks,bryan-nv,bumpers-ar,burns-mt,byrd-wv,campbell-co,chafee-ri,cleland-ga,coats-in,cochran-ms,collins-me,conrad-nd,coverdell-ga,craig-id,damato-ny,daschle-sd,dewine-oh,dodd-ct,domenici-nm,dorgan-nd,durbin-il,enzi-wy,faircloth-nc,feingold-wi,feinstein-ca,ford-ky,frist-tn,glenn-oh,gorton-wa,graham-fl,gramm-tx,grams-mn,grassley-ia,gregg-nh,hagel-ne,harkin-ia,hatch-ut,helms-nc,hollings-sc,hutchinson-ar,hutchison-tx,inhofe-ok,inouye-hi,jeffords-vt,johnson-sd,kempthorne-id,kennedy-ma,kerrey-ne,kerry-ma,kohl-wi,kyl-az,landrieu-la,lautenberg-nj,leahy-vt,levin-mi,lieberman-ct,lott-ms,lugar-in,mack-fl,mccain-az,mcconnell-ky,mikulski-md,moseleybraun-il,moynihan-ny,murkowski-ak,murray-wa,nickles-ok,reed-ri,reid-nv,robb-va,roberts-ks,rockefeller-wv,roth-de,santorum-pa,sarbanes-md,sessions-al,shelby-al,smith-nh,smith-or,snowe-me,specter-pa,stevens-ak,thomas-wy,thompson-tn,thurmond-sc,torricelli-nj,warner-va,wellstone-mn,wyden-or
0,abraham-mi,1.000000,0.112869,0.136168,0.222770,0.134141,0.123907,0.158303,0.143244,0.127530,0.162881,0.106249,0.140346,0.108282,0.126602,0.123742,0.151738,0.078532,0.125201,0.135361,0.175003,0.087042,0.126784,0.117626,0.158389,0.160310,0.118495,0.210736,0.176772,0.193032,0.102856,0.152840,0.185463,0.128237,0.145610,0.181447,0.218029,0.075938,0.165500,0.156500,0.142731,0.148300,0.146812,0.179198,0.143164,0.110481,0.135706,0.136606,0.207545,0.050482,0.165648,0.258783,0.129574,0.095932,0.093540,0.139071,0.118411,0.135085,0.224905,0.144719,0.171199,0.152855,0.196718,0.098884,0.139862,0.191531,0.259811,0.258826,0.186733,0.127116,0.152569,0.254194,0.143750,0.153657,0.132892,0.200711,0.141704,0.176935,0.107303,0.174815,0.117775,0.135270,0.126273,0.164989,0.102136,0.178014,0.113388,0.159714,0.098766,0.161620,0.111183,0.167588,0.135673,0.056847,0.113687,0.139094,0.083433,0.175991,0.097204,0.162597,0.139492
1,akaka-hi,0.112869,1.000000,0.075453,0.078826,0.109133,0.073202,0.064699,0.108374,0.070092,0.101515,0.079190,0.051575,0.070959,0.090988,0.085608,0.080040,0.050489,0.102484,0.084109,0.070745,0.056415,0.055630,0.068405,0.044922,0.104363,0.060923,0.137096,0.073775,0.091576,0.052401,0.080564,0.078229,0.063998,0.074877,0.085277,0.110003,0.098905,0.079206,0.112095,0.103805,0.093976,0.050097,0.079194,0.070912,0.044968,0.061148,0.073818,0.090675,0.034393,0.096819,0.083410,0.063660,0.046898,0.367874,0.086635,0.074574,0.138343,0.102868,0.074207,0.086233,0.058906,0.075305,0.063400,0.066233,0.081747,0.068069,0.128439,0.086763,0.076385,0.074787,0.171675,0.059545,0.075109,0.061542,0.111407,0.174446,0.113375,0.047114,0.081079,0.073563,0.070318,0.061280,0.095037,0.054312,0.047152,0.069558,0.086855,0.051625,0.059170,0.058635,0.091361,0.077797,0.063710,0.130197,0.075711,0.046238,0.106356,0.046031,0.077722,0.067843
2,allard-co,0.136168,0.075453,1.000000,0.096285,0.089875,0.097429,0.110033,0.116389,0.087289,0.086955,0.066927,0.076150,0.078450,0.090597,0.084480,0.099059,0.162403,0.089549,0.092877,0.092678,0.077663,0.067892,0.086660,0.075839,0.124364,0.085107,0.137095,0.078923,0.105982,0.084675,0.111564,0.095517,0.131119,0.099127,0.116429,0.099702,0.048927,0.081861,0.114084,0.095563,0.090110,0.081970,0.135867,0.084602,0.070411,0.138622,0.087637,0.115318,0.040959,0.123221,0.096576,0.076025,0.076996,0.063882,0.079160,0.078764,0.082895,0.103009,0.107424,0.117813,0.076138,0.158231,0.055835,0.106362,0.088577,0.098479,0.145382,0.141676,0.085377,0.099467,0.178677,0.103749,0.098914,0.077551,0.118817,0.097190,0.082444,0.079914,0.102951,0.079300,0.099673,0.096056,0.085240,0.092999,0.070413,0.093418,0.078393,0.085117,0.081972,0.075444,0.102453,0.094339,0.045962,0.125331,0.107462,0.057723,0.104104,0.060701,0.090264,0.081675
3,ashcroft-mo,0.222770,0.078826,0.096285,1.000000,0.120479,0.120272,0.226275,0.180542,0.159450,0.160070,0.090807,0.185773,0.085601,0.111861,0.142458,0.149835,0.080298,0.102020,0.141938,0.187907,0.09

In [29]:
# Get most similar senator to Biden
df_similarity_nobiden = df_similarity[df_similarity['senator']!= 'biden-de']
df_similarity_nobiden[df_similarity_nobiden['biden-de'] == df_similarity_nobiden['biden-de'].max()][['senator','biden-de']]

filename,senator,biden-de
81,roberts-ks,0.393141


The most similar senator to Biden based on the speech is: Patrick Roberts, Republican senator of Kansas. The similarity between the speeches of the two senators is:  0.393141.

Let's check the 10 most similar senators to Biden.

In [30]:
df_similarity_nobiden[['senator','biden-de']].sort_values('biden-de', ascending = False)[['senator','biden-de']].head(10)

filename,senator,biden-de
81,roberts-ks,0.393141
66,lieberman-ct,0.326581
89,smith-or,0.278569
68,lugar-in,0.276504
61,kyl-az,0.275706
26,daschle-sd,0.261798
51,hutchison-tx,0.256332
7,bingaman-nm,0.253516
83,roth-de,0.248345
70,mccain-az,0.244909


Based on their Wikipedia, from the 10 senators 7 are Republican and 3 are Democrat:
* Republicans: Roberts, Smith, Lugar, Hutchison, Roth, Mccain, Kyl
* Democrats: Lieberman, Daschle, Bingaman
    
The results seem pretty counter-intuitive but what we are doing is checking out word frequencies here. If both sides are touching same topics of the agenda, then the speeches aren’t that much different. Maybe some particular wordings can be different, but getting lost because of the topic-related words.  
Roberts senator from the state of Kansas might be on the top, because he talks about the same topics as Biden and their use of words aren’t that much different. A sentiment analysis could bring light on the differences by checking negative or positive sentiments attached to the same topic. Or some other approach leaving bag of words behind and care about text structure.  

We can also experiment with being more strict on the TF-IDF vectorizer's parameters, excluding only 5-5 percentiles is relatively generous. By increasing the bottom threshold, we implicitly bring the topics closer, by decreasing the upper threshold, we rather exclude same topics. It's a delicate trade-off as the calibration affects other words of interest too, not only the topic-specific ones.  

## 4. Data preprocessing and lemmatization, applying TF-IDF vectorizer, calculaiong cosine similarity

In order to compare our results we chose to use lemmatization instead of using stemming. Lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters. In out case (speeches), the context and meaning of the words do matter. 

In [31]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.corpus import wordnet

#### 4.1. Text preprocesser - Lemmatization

In [32]:
# Lemmatizer

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def text_preprocesser_lemmatize(text):
    
    # Remove special characters
    text = re.sub(r'\W',' ', text)
    
    # Remove not alphabet characters
    text = re.sub("[^a-zA-Z]+", " ", text)
    
    # Lowercase and tokenize
    tokens = [word.lower() for word in nltk.word_tokenize(text)]
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Remove words with length less than 3 characters
    tokens = [token for token in tokens if len(token)>=3]
    
    lemma = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
    # Join
    preprocessed_text = ' '.join(lemma)

    return preprocessed_text


In [33]:
# Check one element
text_preprocesser_lemmatize(df['speech'].iloc[0])[:300]

'abraham president debate final passage omnibus appropriation bill american competitiveness workforce improvement act include title subdivision ask unanimous consent number document print record include two document receive administration negotiation whose inclusion seek help illuminate meaning provi'

#### 4.2. Apply text preprocesser

As it's a time-consuming process, the result were saved, then read back. Please uncomment the code chunk for testing the code. 

In [34]:
#### Apply to df

# df_preprocessed_lemmatize = df['speech'].apply(text_preprocesser_lemmatize)

#### Save

# import pickle
# with open(f'df_preprocessed_lemmatize.p', 'wb') as f:
        # pickle.dump(df_preprocessed_lemmatize, f)

In [35]:
with open(f'df_preprocessed_lemmatize.p', 'rb') as data:
    df_preprocessed_lemmatize = pickle.load(data)
df_preprocessed_lemmatize[0][:300]

'abraham president debate final passage omnibus appropriation bill american competitiveness workforce improvement act include title subdivision ask unanimous consent number document print record include two document receive administration negotiation whose inclusion seek help illuminate meaning provi'

In [36]:
len(df_preprocessed_lemmatize)

100

#### 4.3. Apply TF-IDF vectorizer

We will:
* ignore terms that appear in less than 5% of the documents and
* ignore terms that appear in more than 95% of the documents

In [37]:
# Apply TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=0.05, max_df= 0.95) # preprocessing was done before
tfidf_lemma = tfidf_vectorizer.fit_transform(df_preprocessed_lemmatize)

In [38]:
# Convert to df
df_tfidf_lemma = pd.DataFrame(tfidf_lemma.toarray().transpose(), index=tfidf_vectorizer.get_feature_names())
df_tfidf_lemma.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
aaa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.006132,0.000000,0.008438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aapcc,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010217,0.000000,0.003070,0.000000,0.004492,0.007736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.024372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016947,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.058109
aaron,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009305,0.002323,0.000000,0.000000,0.002016,0.000000,0.000000,0.001377,0.000000,0.001699,0.000000,0.000000,0.001969,0.000000,0.000000,0.000000,0.000000,0.001679,0.0,0.002691,0.000000,0.006164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001489,0.003029,0.004037,0.000901,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001389,0.000000,0.001583,0.000000,0.000000,0.000000,0.003357,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013859,0.004107,0.002589,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aarp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002121,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002196,0.000000,0.000000,0.034470,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002560,0.000000,0.005643,0.000000,0.011853,0.002925,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001903,0.000000,0.000000,0.000000,0.003577,0.000000,0.004631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000955,0.000000,0.000000,0.000000,0.002472,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005393,0.000000,0.003375,0.000000,0.002425,0.000000,0.0,0.000000,0.000000,0.002116,0.000000,0.004696,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aba,0.004618,0.000000,0.000000,0.004740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [39]:
# Assign column names to matrix
df_tfidf_lemma.columns = senator_names
df_tfidf_lemma.head(10)

filename,abraham-mi,akaka-hi,allard-co,ashcroft-mo,baucus-mt,bennett-ut,biden-de,bingaman-nm,bond-mo,boxer-ca,breaux-la,brownback-ks,bryan-nv,bumpers-ar,burns-mt,byrd-wv,campbell-co,chafee-ri,cleland-ga,coats-in,cochran-ms,collins-me,conrad-nd,coverdell-ga,craig-id,damato-ny,daschle-sd,dewine-oh,dodd-ct,domenici-nm,dorgan-nd,durbin-il,enzi-wy,faircloth-nc,feingold-wi,feinstein-ca,ford-ky,frist-tn,glenn-oh,gorton-wa,graham-fl,gramm-tx,grams-mn,grassley-ia,gregg-nh,hagel-ne,harkin-ia,hatch-ut,helms-nc,hollings-sc,hutchinson-ar,hutchison-tx,inhofe-ok,inouye-hi,jeffords-vt,johnson-sd,kempthorne-id,kennedy-ma,kerrey-ne,kerry-ma,kohl-wi,kyl-az,landrieu-la,lautenberg-nj,leahy-vt,levin-mi,lieberman-ct,lott-ms,lugar-in,mack-fl,mccain-az,mcconnell-ky,mikulski-md,moseleybraun-il,moynihan-ny,murkowski-ak,murray-wa,nickles-ok,reed-ri,reid-nv,robb-va,roberts-ks,rockefeller-wv,roth-de,santorum-pa,sarbanes-md,sessions-al,shelby-al,smith-nh,smith-or,snowe-me,specter-pa,stevens-ak,thomas-wy,thompson-tn,thurmond-sc,torricelli-nj,warner-va,wellstone-mn,wyden-or
aaa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.006132,0.000000,0.008438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aapcc,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010217,0.000000,0.003070,0.000000,0.004492,0.007736,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.024372,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016947,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.058109
aaron,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009305,0.002323,0.000000,0.000000,0.002016,0.000000,0.000000,0.001377,0.000000,0.001699,0.000000,0.000000,0.001969,0.000000,0.000000,0.000000,0.000000,0.001679,0.0,0.002691,0.000000,0.006164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001489,0.003029,0.004037,0.000901,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001389,0.000000,0.001583,0.000000,0.000000,0.000000,0.003357,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013859,0.004107,0.002589,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
aarp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002121,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002196,0.000000,0.000000,0.034470,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002560,0.000000,0.0056

Note that the number of tokens decreased by about 6000 terms compared to using the raw tokenized dataset without lemmatization.

In [40]:
len(df_tfidf_lemma) - len(df_tfidf)

-6255

#### 4. Apply cosine similarity

In [41]:
# Reshape 
for i in range(1, num_col+1):
    globals()["txt" + str(i)] = df_tfidf_lemma[df_tfidf_lemma.columns[i-1]].values.reshape(1, -1) 

In [42]:
# Biden
txt7

array([[0.        , 0.        , 0.        , ..., 0.02007284, 0.        ,
        0.00170392]])

In [43]:
# Similarity between Biden and Roberts - sample
print("Similarity Biden and Roberts:", cosine_similarity(txt7, txt82))

Similarity Biden and Roberts: [[0.47130203]]


In [44]:
# Reshape to vector
import numpy as np
for i in range(1, num_col+1):
    globals()["txt" + str(i)] = np.squeeze(np.asarray(globals()["txt" + str(i)]))

In [45]:
txt7

array([0.        , 0.        , 0.        , ..., 0.02007284, 0.        ,
       0.00170392])

In [46]:
# Create list of vectors
list_of_txts = []
for i in range(1, num_col+1):
    element = "txt" + str(i)
    list_of_txts.append(element)
print(list_of_txts)

['txt1', 'txt2', 'txt3', 'txt4', 'txt5', 'txt6', 'txt7', 'txt8', 'txt9', 'txt10', 'txt11', 'txt12', 'txt13', 'txt14', 'txt15', 'txt16', 'txt17', 'txt18', 'txt19', 'txt20', 'txt21', 'txt22', 'txt23', 'txt24', 'txt25', 'txt26', 'txt27', 'txt28', 'txt29', 'txt30', 'txt31', 'txt32', 'txt33', 'txt34', 'txt35', 'txt36', 'txt37', 'txt38', 'txt39', 'txt40', 'txt41', 'txt42', 'txt43', 'txt44', 'txt45', 'txt46', 'txt47', 'txt48', 'txt49', 'txt50', 'txt51', 'txt52', 'txt53', 'txt54', 'txt55', 'txt56', 'txt57', 'txt58', 'txt59', 'txt60', 'txt61', 'txt62', 'txt63', 'txt64', 'txt65', 'txt66', 'txt67', 'txt68', 'txt69', 'txt70', 'txt71', 'txt72', 'txt73', 'txt74', 'txt75', 'txt76', 'txt77', 'txt78', 'txt79', 'txt80', 'txt81', 'txt82', 'txt83', 'txt84', 'txt85', 'txt86', 'txt87', 'txt88', 'txt89', 'txt90', 'txt91', 'txt92', 'txt93', 'txt94', 'txt95', 'txt96', 'txt97', 'txt98', 'txt99', 'txt100']


In [47]:
# Create similarity matrix
from scipy import sparse
A =  np.array(np.array([eval(name) for name in list_of_txts]))
A_sparse = sparse.csr_matrix(A)
similarities = cosine_similarity(A_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[1.         0.11048625 0.12188617 ... 0.08611824 0.15113374 0.12806296]
 [0.11048625 1.         0.06546759 ... 0.04187035 0.07262217 0.06461585]
 [0.12188617 0.06546759 1.         ... 0.05451684 0.08047845 0.07196163]
 ...
 [0.08611824 0.04187035 0.05451684 ... 1.         0.07501323 0.07624209]
 [0.15113374 0.07262217 0.08047845 ... 0.07501323 1.         0.12278158]
 [0.12806296 0.06461585 0.07196163 ... 0.07624209 0.12278158 1.        ]]



In [48]:
# Define similarity matrix
df_similarity = pd.DataFrame(similarities)
df_similarity.columns = senator_names
df_similarity.insert(0,'senator',senator_names)
df_similarity.head()

filename,senator,abraham-mi,akaka-hi,allard-co,ashcroft-mo,baucus-mt,bennett-ut,biden-de,bingaman-nm,bond-mo,boxer-ca,breaux-la,brownback-ks,bryan-nv,bumpers-ar,burns-mt,byrd-wv,campbell-co,chafee-ri,cleland-ga,coats-in,cochran-ms,collins-me,conrad-nd,coverdell-ga,craig-id,damato-ny,daschle-sd,dewine-oh,dodd-ct,domenici-nm,dorgan-nd,durbin-il,enzi-wy,faircloth-nc,feingold-wi,feinstein-ca,ford-ky,frist-tn,glenn-oh,gorton-wa,graham-fl,gramm-tx,grams-mn,grassley-ia,gregg-nh,hagel-ne,harkin-ia,hatch-ut,helms-nc,hollings-sc,hutchinson-ar,hutchison-tx,inhofe-ok,inouye-hi,jeffords-vt,johnson-sd,kempthorne-id,kennedy-ma,kerrey-ne,kerry-ma,kohl-wi,kyl-az,landrieu-la,lautenberg-nj,leahy-vt,levin-mi,lieberman-ct,lott-ms,lugar-in,mack-fl,mccain-az,mcconnell-ky,mikulski-md,moseleybraun-il,moynihan-ny,murkowski-ak,murray-wa,nickles-ok,reed-ri,reid-nv,robb-va,roberts-ks,rockefeller-wv,roth-de,santorum-pa,sarbanes-md,sessions-al,shelby-al,smith-nh,smith-or,snowe-me,specter-pa,stevens-ak,thomas-wy,thompson-tn,thurmond-sc,torricelli-nj,warner-va,wellstone-mn,wyden-or
0,abraham-mi,1.000000,0.110486,0.121886,0.214059,0.123617,0.110593,0.151989,0.129213,0.150210,0.157791,0.095588,0.134765,0.105546,0.119687,0.120104,0.141327,0.072466,0.116841,0.125299,0.176683,0.079305,0.121118,0.108866,0.136141,0.145726,0.105709,0.199346,0.168373,0.177827,0.091320,0.141605,0.179400,0.110488,0.139630,0.166749,0.214007,0.069387,0.152558,0.152071,0.127789,0.138701,0.131590,0.166242,0.129377,0.099104,0.126492,0.121284,0.196111,0.071877,0.151861,0.265876,0.116147,0.091634,0.088810,0.124504,0.112484,0.129560,0.203818,0.130722,0.156337,0.143031,0.195537,0.097383,0.128833,0.185429,0.254233,0.250136,0.177074,0.117660,0.140678,0.239406,0.131559,0.146161,0.122846,0.187485,0.130099,0.173255,0.106051,0.158631,0.113034,0.126090,0.132755,0.148182,0.089159,0.187348,0.099303,0.200946,0.090585,0.158143,0.105178,0.159254,0.126848,0.049765,0.108563,0.120178,0.076067,0.161172,0.086118,0.151134,0.128063
1,akaka-hi,0.110486,1.000000,0.065468,0.072664,0.098570,0.064234,0.063162,0.093485,0.086077,0.094349,0.078915,0.048435,0.070707,0.070815,0.077662,0.077670,0.044663,0.101087,0.083577,0.066125,0.052789,0.051590,0.070232,0.041102,0.101871,0.052757,0.133835,0.065665,0.082087,0.046288,0.081157,0.073214,0.057167,0.072905,0.073599,0.105232,0.102397,0.068755,0.109340,0.095666,0.088423,0.047279,0.074221,0.066862,0.039577,0.055898,0.071746,0.083275,0.050783,0.093374,0.078034,0.061413,0.047155,0.386910,0.078851,0.074785,0.135439,0.087925,0.068755,0.079414,0.053345,0.075361,0.059408,0.060620,0.079275,0.059731,0.115228,0.083935,0.077414,0.060933,0.160780,0.054928,0.070322,0.054763,0.105119,0.156191,0.103375,0.043680,0.071026,0.068767,0.068111,0.070587,0.092647,0.048896,0.039737,0.060879,0.102930,0.048344,0.052164,0.056230,0.086882,0.071064,0.059385,0.084238,0.067905,0.043477,0.096029,0.041870,0.072622,0.064616
2,allard-co,0.121886,0.065468,1.000000,0.092392,0.078192,0.080981,0.101882,0.103268,0.108509,0.076696,0.058329,0.069460,0.075152,0.074783,0.073945,0.086406,0.164342,0.083928,0.084419,0.084638,0.077257,0.059844,0.083738,0.070937,0.111811,0.075138,0.133717,0.070808,0.091461,0.076130,0.101601,0.081377,0.121183,0.091644,0.097902,0.088859,0.042602,0.070726,0.105416,0.084704,0.082921,0.072132,0.124386,0.073762,0.063567,0.125935,0.077832,0.101616,0.057995,0.112364,0.088149,0.068559,0.071576,0.065584,0.068781,0.075228,0.074843,0.086466,0.097245,0.101952,0.068368,0.147341,0.048064,0.098142,0.078851,0.085456,0.131475,0.132196,0.076805,0.086286,0.165757,0.090330,0.093694,0.067951,0.109765,0.081287,0.071838,0.074081,0.086208,0.071823,0.099806,0.096207,0.075495,0.085392,0.061778,0.080197,0.093924,0.081228,0.072344,0.070311,0.087425,0.085684,0.037038,0.104308,0.094633,0.051181,0.091434,0.054517,0.080478,0.071962
3,ashcroft-mo,0.214059,0.072664,0.092392,1.000000,0.111123,0.104321,0.226768,0.163228,0.195251,0.153874,0.084892,0.166423,0.079495,0.100717,0.146820,0.140848,0.077476,0.093370,0.134395,0.181675,0.08

In [49]:
# Get most similar senator to Biden
df_similarity_nobiden = df_similarity[df_similarity['senator']!= 'biden-de']
df_similarity_nobiden[df_similarity_nobiden['biden-de'] == df_similarity_nobiden['biden-de'].max()][['senator','biden-de']]

filename,senator,biden-de
81,roberts-ks,0.471302


With the lemmatization the most similar senator to Biden based on the speech is still Patrick Roberts, Republican senator of Kansas. However, the similarity index between the two senators' speeches slightly increased from 0.393141 to 0.471302.

Let's ceheck the 10 most similar senators to Biden.

In [50]:
df_similarity_nobiden[['senator','biden-de']].sort_values('biden-de', ascending = False).head(10)

filename,senator,biden-de
81,roberts-ks,0.471302
66,lieberman-ct,0.326881
89,smith-or,0.277172
68,lugar-in,0.274742
26,daschle-sd,0.260803
61,kyl-az,0.257698
51,hutchison-tx,0.252216
7,bingaman-nm,0.248942
83,roth-de,0.244127
72,mikulski-md,0.238297


The picture is very similar to the simple tokenization case. The difference is that with lemmatization in the top 10 most similar senator we have 6 Republicans and 4 Democrats. Instead of McCain Democrat senator Mikulski appeared rank as the 10th most similar senator to Biden. 